In [1]:
import numpy as np
import math
from typing import Callable, Union, Tuple, List, Any
import pandas as pd
pd.set_option("display.precision", 8)

# Initial-Value Problems for Ordinary Differential Equations

## Chapter 5.2 Euler's Method
Given 
$$ \frac{dy}{dt} = f(t, y),\,\,\, a\leq t\leq b,\,\,\,y(a)=\alpha $$
and consider
$$ t_i = a+ih,\,\,\,i=0,1,2\cdots,N$$
Suppose $y(t)$ has two continuous derivatives on $[a, b]$, then by Taylor's Theorem
$$ y(t_{i+1})=y(t_i)+(t_{i+1}-t_i)y'(t_i)+\frac{(t_{i+1}-t_i)^2}{2}y''(\xi_i) $$
because $h=t_{i+1}-t_i$, then
$$ y(t_{i+1})=y(t_i)+hf(t_i,y(t_i))+\frac{h^2}{2}y''(\xi_i)$$

The definition of $t_i$ and $h$ are the same throughout the context.

Euler's method constructs $w_i\approx y(t_i)$, by deleting the remainder term. In short,
$$w_0=\alpha$$
$$w_{i+1}=w_i+hf(t_i, w_i),\,\,\,i=0, 1,\cdots,N-1$$

In [2]:
def Euler(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
          a:Union[float, int],
          b:Union[float, int],
          alpha:Union[float, int],
          h:Union[float, int]=None,
          N:int=None,
         ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * alpha
    for idx in range(1, N + 1):
        w_[idx] =  w_[idx - 1] + h * f(t_[idx - 1], w_[idx - 1])
    return (t_, w_)

In [3]:
f = lambda t, y: y - t ** 2 + 1
Euler(f, 0, 2, 0.5, h=0.5)

(array([0. , 0.5, 1. , 1.5, 2. ]),
 array([0.5   , 1.25  , 2.25  , 3.375 , 4.4375]))

In [4]:
f = lambda t, y: np.cos(2 * t) + np.sin(3 * t)
Euler(f, 0, 1, 1, N=4)

(array([0.  , 0.25, 0.5 , 0.75, 1.  ]),
 array([1.        , 1.25      , 1.63980533, 2.02425465, 2.23645725]))

## Chapter 5.3 Higher-Order Taylor Methods
Suppose the solution $y(t)$ to the initial-value problem
$$ \frac{dy}{dt} = f(t, y),\,\,\, a\leq t\leq b,\,\,\,y(a)=\alpha $$
has $(n+1)$ continuous derivatives. By its $n$th Taylor polynomial about $t_i$ and evaluate at $t_{i+1}$,
$$y(t_{{i+1}})=y(t_i)+hy'(t_i)+\frac{h^2}{2}y''(t_i)+\cdots+\frac{h^n}{n!}y^{(n)}(t_i)+\frac{h^{n+1}}{(n+1)!}y^{(n+1)}(\xi_i)$$
for some $\xi_i\in(t_i, t_{i+1})$
Successive differentiation of the solution, $y(t)$ gives
$$y'(t)=f(t, y(t)),\,\,\,y''(t)=f'(t, y(t)),\,\,\,\text{ and generally, }\,\,\, y^{(k)}(t)=f^{(k-1)}(t, y(t))$$
Substituting these findings into previous equations gives
$$y(t_{{i+1}})=y(t_i)+hf(t_i)+\frac{h^2}{2}f'(t_i)+\cdots+\frac{h^n}{n!}f^{(n - 1)}(t_i)+\frac{h^{n+1}}{(n+1)!}f^{(n)}(\xi_i)$$
Similar to Euler's method, in short
$$w_0=\alpha$$
$$w_{i+1}=w_i+hT^{(n)}(t_i, w_i),\,\,\,i=0,1,\cdots,N-1$$
where
$$T^{(n)}(t_i, w_i)=\sum^{n-1}_{i=0}\frac{h^{i}}{(i+1)!}f^{(i)}(t_i, w_i)$$
Euler's method is Taylor's method of order one.

In [5]:
def Taylor(func:List[Callable[[Union[float, int], Union[float, int]], Union[float, int]]],
           a:Union[float, int],
           b:Union[float, int],
           alpha:Union[float, int],
           h:Union[float, int]=None,
           N:int=None
          ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * alpha
    
    for idx in range(1, N + 1):
        T = sum([(h ** jdx) * f(t_[idx - 1], w_[idx - 1]) / math.factorial(jdx + 1) for jdx, f in enumerate(func)])
        w_[idx] = w_[idx - 1] + h * T
    return (t_, w_)

In [6]:
f = [
    lambda t, y: y - t ** 2 + 1,
    lambda t, y: y - t ** 2 + 1 - 2 * t
]
Taylor(f, 0, 2, 0.5, 0.2)

(array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. ]),
 array([0.5       , 0.83      , 1.2158    , 1.652076  , 2.13233272,
        2.64864592, 3.19134802, 3.74864458, 4.30614639, 4.8462986 ,
        5.34768429]))

## Chapter 5.4 Runge-Kutta Methods
__Runge-Kutta methods__ have the high-order local truncation error of the Taylor methods
but eliminate the need to compute and evaluate the derivatives of $f (t, y)$

In general, __Runge-Kutta Methods__ have the following form:
$$
y_{i+1} = y_i+h\sum_{i=1}^{s}b_ik_i
$$
where
$y_i=y(t_i)$

$k_1=f(t_i,y_i)$

$k_2=f(t_i+c_2h,y_i+(a_{21}k_1)h)$

$k_3=f(t_i+c_3h,y_i+(a_{31}k_1+a_{32}k_2)h)$

$\vdots$

$k_s=f(t_i+c_sh,y_i+(a_{s1}k_1+a_{s2}k_2+\cdots+a_{s, s-1}k_{s-1})h)$

The parameters above can be store as an matrix where the illustrated as below:
\begin{bmatrix}
0 & 0 & 0 & \cdots & 0\\
c_2 & a_{21} & a_{22} & \cdots & a_{s2}\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
c_s & a_{s1} & a_{s2} & \cdots & a_{ss}\\
0 & b_1 & b_2 & \cdots & b_s
\end{bmatrix}	

The matrix $[a_{ij}]$ is called the Runge-Kutta matrix while $b_i$ and $c_i$ are known as the weights and the nodes.

Refer to [Wikipedia](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) for more information

### Runge-Kutta Methods of Order Two
There is a family of such methods, parameterized by α and given by the formula
$$
y(t_{i+1})=y(t_i)+h\biggl(\biggl(1-\frac{1}{2\alpha}f(t_i, y(t_i))\biggr)+\frac{1}{2\alpha}f(t_i+\alpha h, y(t_i)+\alpha f(t_i, y(t_i)))\biggr)
$$
where 
$\alpha=\frac{1}{2}$ gives the midpoint method, $\alpha=1$ is Modified Euler Method(Heun's method), and $\alpha=\frac{2}{3}$ is Ralston's method.

So by the matrix notation introduced above, we have
\begin{bmatrix}
0 & 0 & 0\\
\alpha & \alpha & 0\\
0 & 1 - \frac{1}{2\alpha} & \frac{1}{2\alpha}
\end{bmatrix}	


#### Midpoint Method
$w_0=\alpha$

$w_{i+1}=w_i+hf\biggl(t_i+\frac{h}{2}, w_i+\frac{h}{2}f(t_i, w_i)\biggr),\,\,\,i=0, 1,\cdots,N-1$

#### Modified Euler Method
$w_0=\alpha$

$w_{i+1}=w_i+\frac{h}{2}[f(t_i, w_i)+f(t_{i+1}, w_i+hf(t_i, w_i))],\,\,\,i=0, 1,\cdots,N-1$

#### Ralston's Method
$w_0=\alpha$

$w_{i+1}=w_i+h\biggl[\frac{1}{4}f(t_i, w_i)+\frac{3}{4}f(t_{i+1}+\frac{2}{3}h, w_i+\frac{2}{3}hf(t_i, w_i))\biggr],\,\,\,i=0, 1,\cdots,N-1$

In [7]:
def RK2_Generic(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
          a:Union[float, int],
          b:Union[float, int],
          y_init:Union[float, int],
          h:Union[float, int, None]=None,
          alpha:float=0.5,
          N:Union[int, None]=None,
         ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * y_init
    for idx in range(1, N + 1):
        w_[idx] =  w_[idx - 1] + h * ((1 - 1 / (2 * alpha)) * f(t_[idx - 1], w_[idx - 1]) + 
                                      (f(t_[idx - 1] + alpha * h, w_[idx - 1] + alpha * h * f(t_[idx - 1], w_[idx - 1]))) / (2 * alpha)
                                      )
    return (t_, w_)

### Runge-Kutta Methods of Order Three
#### Heun's Method

$w_0=\alpha$

$w_{i+1}=w_i+\frac{h}{4}\biggl[f(t_i, w_i)+3f(t_{i}+\frac{2}{3}h, w_i+\frac{2}{3}hf(t_i+\frac{1}{3}h, w_i+\frac{1}{3}f(t_i, w_i)))\biggr],\,\,\,i=0, 1,\cdots,N-1$

In [8]:
def Heun(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
          a:Union[float, int],
          b:Union[float, int],
          y_init:Union[float, int],
          h:Union[float, int]=None,
          N:int=None,
         ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * y_init
    for idx in range(1, N + 1):
        w_[idx] =  w_[idx - 1] + h * (f(t_[idx - 1], w_[idx - 1]) + 3 * f(t_[idx - 1] + (2 / 3) * h, w_[idx - 1] + (2 / 3) * 
                                        h * f(t_[idx - 1] + (1 / 3) * h, w_[idx - 1] + (1 / 3) * h *  f(t_[idx - 1], w_[idx - 1])))) / 4
    return (t_, w_)

### Runge-Kutta Methods of Order Four (Most commonly used)

$w_0=\alpha$

$k_1=hf(t_i, w_i)$

$k_2=hf\biggl(t_i+\frac{h}{2}, w_i+\frac{1}{2}k_1\biggr)$

$k_3=hf\biggl(t_i+\frac{h}{2}, w_i+\frac{1}{2}k_2\biggr)$

$k_4=hf(t_{i+1}, w_i+k_3)$

$w_{i+1}=\frac{1}{6}(k_1+2k_2+2k_3+k_4)$

for each $i=0, 1,\cdots,N-1$

In [9]:
def RK4(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
        a:Union[float, int],
        b:Union[float, int],
        y_init:Union[float, int],
        h:Union[float, int]=None,
        N:int=None,
        ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * y_init
    for idx in range(1, N + 1):
        k_1 = h * f(t_[idx - 1], w_[idx - 1])
        k_2 = h * f(t_[idx - 1] + h / 2, w_[idx - 1] + k_1 / 2)
        k_3 = h * f(t_[idx - 1] + h / 2, w_[idx - 1] + k_2 / 2)
        k_4 = h * f(t_[idx], w_[idx - 1] + k_3)
        w_[idx] = w_[idx - 1] + (k_1 + 2 * k_2 + 2 * k_3 + k_4) / 6
    return (t_, w_)

#### Implementation of General Runge-Kutta Methods
In convinient, the implementation of the function will take $[a_{ij}]$ as a numpy matrix while $b_i$ and $c_i$ will be 1-D numpy array known as parameter _rk_mat_, _weights_, _nodes_ respectively

In [10]:
def RK(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
       a:Union[float, int],
       b:Union[float, int],
       rk_mat:np.ndarray[Union[float, int], Any],
       weights:np.ndarray[Union[float, int], Any],
       nodes:np.ndarray[Union[float, int], Any],
       y_init:Union[float, int],
       h:Union[float, int]=None,
       N:int=None,
        ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    if rk_mat.shape[0] != nodes.shape[0]:
        raise ValueError("The number of row of rk_mat not equal to the size of nodes...")
    if rk_mat.shape[1] != weights.shape[0]:
        raise ValueError("The number of column of rk_mat not equal to the size of weights...")
    
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * y_init
    for idx in range(1, N + 1):
        k_ = np.zeros((weights.shape[0], ), dtype=float)
        k_[0] = f(t_[idx - 1], w_[idx - 1])
        for jdx in range(1, weights.shape[0]):
            k_[jdx] += f(t_[idx - 1] + nodes[jdx - 1] * h, w_[idx - 1] + (rk_mat[jdx - 1] @ k_) * h)
        w_[idx] =  w_[idx - 1] + h * np.sum(k_ * weights)
    return (t_, w_)
    

In [11]:
f = lambda t, y: y - t ** 2 + 1

t_1, w_1 = RK2_Generic(f, 0, 2, 0.5, 0.2, alpha=0.5)
t_2, w_2 = RK2_Generic(f, 0, 2, 0.5, 0.2, alpha=1)
t_3, w_3 = RK2_Generic(f, 0, 2, 0.5, 0.2, alpha=2 / 3)
t_4, w_4 = Heun(f, 0, 2, 0.5, 0.2)
t_5, w_5 = RK4(f, 0, 2, 0.5, 0.2)


midpoint_rk = np.array([[0.5, 0]])
midpoint_c = np.array([0.5])
midpoint_b = np.array([0, 1])

modified_euler_rk = np.array([[1, 0]])
modified_euler_c = np.array([1])
modified_euler_b = np.array([0.5, 0.5])

ralston_rk = np.array([[2 / 3, 0]])
ralston_c = np.array([2 / 3])
ralston_b = np.array([1 / 4, 3 / 4])

heun3_rk = np.array([[1 / 3, 0, 0], [0, 2 / 3, 0]])
heun3_c = np.array([1 / 3, 2 / 3])
heun3_b = np.array([1 / 4, 0, 3 / 4])

RK4_rk = np.array([[1 / 2, 0, 0, 0], [0, 1 / 2, 0, 0], [0, 0, 1, 0]])
RK4_c = np.array([1 / 2, 1 / 2, 1])
RK4_b = np.array([1 / 6, 1 / 3, 1 / 3, 1 / 6])

t_6, w_6 = RK(f, 0, 2, midpoint_rk, midpoint_b, midpoint_c, 0.5, 0.2)
t_7, w_7 = RK(f, 0, 2, modified_euler_rk, modified_euler_b, modified_euler_c, 0.5, 0.2)
t_8, w_8 = RK(f, 0, 2, ralston_rk, ralston_b, ralston_c, 0.5, 0.2)
t_9, w_9 = RK(f, 0, 2, heun3_rk, heun3_b, heun3_c, 0.5, 0.2)
t_10, w_10 = RK(f, 0, 2, RK4_rk, RK4_b, RK4_c, 0.5, 0.2)


# The methods with _ at the end means that the values computed using
# the general formula
pd.DataFrame(
    {
        "t":t_1,
        "Midpoint":w_1,
        "Midpoint_":w_5,
        "Modified Euler":w_2,
        "Modified Euler_":w_6,
        "Ralston":w_3,
        "Ralston_":w_7,
        "Heun":w_4,
        "Heun_":w_8,
        "RK4":w_5,
        "RK4_":w_10
    }
)


,t,Midpoint,Midpoint_,Modified Euler,Modified Euler_,Ralston,Ralston_,Heun,Heun_,RK4,RK4_
0,0.0,0.50000000,0.50000000,0.50000000,0.50000000,0.50000000,0.50000000,0.50000000,0.50000000,0.50000000,0.50000000
1,0.2,0.82800000,0.82929333,0.82600000,0.82800000,0.82733333,0.82600000,0.82924444,0.82733333,0.82929333,0.82929333
2,0.4,1.21136000,1.21407621,1.20692000,1.21136000,1.20988000,1.20692000,1.21397499,1.20988000,1.21407621,1.21407621
3,0.6,1.64465920,1.64892202,1.63724240,1.64465920,1.64218693,1.63724240,1.64876590,1.64218693,1.64892202,1.64892202
4,0.8,2.12128422,2.12720268,2.11023573,2.12128422,2.11760139,2.11023573,2.12699053,2.11760139,2.12720268,2.12720268
5,1.0,2.63316675,2.64082269,2.61768759,2.63316675,2.62800703,2.61768759,2.64055555,2.62800703,2.64082269,2.64082269
6,1.2,3.17046344,3.17989417,3.14957886,3.17046344,3.16350191,3.14957886,3.17957629,3.16350191,3.17989417,3.17989417
7,1.4,3.72116540,3.73234007,3.69368621,3.72116540,3.71200567,3.69368621,3.73198028,3.71200567,3.73234007,3.73234007
8,1.6,4.27062178,4.28340950,4.23509717,4.27062178,4.25878025,4.23509717,4.28302303,4.25878025,4.28340950,4.28340950
9,1.8,4.80095857,4.81508569,4.75561855,4.80095857,4.78584523,4.75561855,4.81469657,4.78584523,4.81508569,4.81508569


## Chapter 5.5 Error Control and the Runge-Kutta-Fehlberg Method

In [12]:
def RKEmbedded(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
            a:Union[float, int],
            b:Union[float, int],
            rk_mat:np.ndarray[Union[float, int], Any],
            weights:np.ndarray[Union[float, int], Any],
            weights_:np.ndarray[Union[float, int], Any],
            nodes:np.ndarray[Union[float, int], Any],
            y_init:Union[float, int],
            h_max:Union[float, int],
            h_min:Union[float, int],
            n:int,
            delta_coef:float,
            tol:float=1e-6,
        ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:

    if rk_mat.shape[0] != nodes.shape[0]:
        raise ValueError("The number of row of rk_mat not equal to the size of nodes...")
    if rk_mat.shape[1] != weights.shape[0]:
        raise ValueError("The number of column of rk_mat not equal to the size of weights...")
    
    h = h_max
    t_, w_ = [a], [y_init]
    t = a
    idx = 1
    while True:
        k_ = np.zeros((weights.shape[0], ), dtype=float)
        k_[0] = f(t, w_[idx - 1])
        for jdx in range(1, weights.shape[0]):
            k_[jdx] += f(t + nodes[jdx - 1] * h, w_[idx - 1] + (rk_mat[jdx - 1] @ k_) * h)
        R = abs((h * np.sum(k_ * weights)) - (h * np.sum(k_ * weights_))) / h
        if R <= tol:
            t = t_[idx - 1] + h
            t_.append(t)
            w_.append(w_[idx - 1] + h * np.sum(k_ * weights))
            idx += 1
        delta = delta_coef * (tol / R) ** (1 / n)
        if delta <= 0.1:
            delta = 0.1
        elif delta >= 4:
            delta = 4
        h *= delta
        h = min(h, h_max)
        if t >= b:
            return (np.array(t_), np.array(w_))
        elif t + h > b:
            h = b - t
        elif h < h_min:
            return (np.array(t_), np.array(w_))
    

In [13]:
RKF_rk = np.array([
    [0.25, 0, 0, 0, 0, 0], [3 / 32, 9 / 32, 0, 0, 0, 0],
    [1932 / 2197, -7200 / 2197, 7296 / 2197, 0, 0, 0],
    [439 / 216, -8, 3680 / 513, -845 / 4104, 0, 0],
    [-8 / 27, 2, -3544 / 2565, 1859 / 4104, -11 / 40, 0]
    ])
RKF_c = np.array([1 / 4, 3 / 8, 12 / 13, 1, 1 / 2])
RKF_bp = np.array([16 / 135, 0, 6656 / 12825, 28561 / 56430, -9 / 50, 2 / 55])
RKF_b = np.array([25 / 216, 0, 1408 / 2565, 2197 / 4104, -1 / 5, 0])

f = lambda t, y: y - t ** 2 + 1
RKEmbedded(f, 0, 2, RKF_rk, RKF_b, RKF_bp, RKF_c, 0.5, 0.25, 0.01, 4, 0.84, 0.00001)

(array([0.       , 0.25     , 0.4865522, 0.7293332, 0.9793332, 1.2293332,
        1.4793332, 1.7293332, 1.9793332, 2.       ]),
 array([0.5       , 0.9204886 , 1.39649101, 1.95374879, 2.58642601,
        3.26046051, 3.95209554, 4.63082682, 5.25748606, 5.30548963]))

## Chapter 5.6 Multistep Methods

In [14]:
# Compute the coefficients of f in the Adams-Bashforth and Adams-Moulton methods

def BackwardDiffCoef(k:int):
    Q = np.zeros((k, k))
    Q[0, 0] = 1
    for idx in range(k - 1):
        for jdx in range(k - 1):
            if Q[idx, jdx] != 0:
                Q[idx, jdx + 1] += 1 * Q[idx, jdx]
                Q[idx + 1, jdx + 1] -= 1 * Q[idx, jdx]
    return Q

def ForwardDiffCoef(k:int):
    Q = np.zeros((k, k))
    Q[0, 0] = 1
    for idx in range(k - 1):
        for jdx in range(k - 1):
            if Q[idx, jdx] != 0:
                Q[idx, jdx + 1] -= 1 * Q[idx, jdx]
                Q[idx + 1, jdx + 1] += 1 * Q[idx, jdx]
    return Q

def CombCoef(m:int, backward:bool=True) -> np.ndarray:
    coef = np.ones((m, ))
    for mdx in range(1, m):
        roots = np.arange(mdx)
        poly = np.polynomial.Polynomial.fromroots(-roots if backward else roots)
        anti_poly = poly.integ()
        coef[mdx] = (anti_poly(1) - anti_poly(0)) / math.factorial(mdx)
    return coef

def MultiStepCoef(m:int, backward:bool=True) -> np.ndarray:
    if backward:
        return (CombCoef(m) * BackwardDiffCoef(m)).sum(axis=1)
    return (CombCoef(m, backward) * ForwardDiffCoef(m)).sum(axis=1)


In [15]:
print("Adam-Bashforth")
for idx in range(2, 6):
    print(MultiStepCoef(idx))
print("Adam-Moulton")
for idx in range(3, 6):
    print(MultiStepCoef(idx, False))

print("Adam-Bashforth Coefficient for LTE")
for idx in range(2, 6):
    print(abs(MultiStepCoef(idx + 1)[-1]))
print("Adam-Moulton Coefficient for LTE")
for idx in range(3, 6):
    print(-abs(MultiStepCoef(idx + 1, False)[-1]))


Adam-Bashforth
[ 1.5 -0.5]
[ 1.91666667 -1.33333333  0.41666667]
[ 2.29166667 -2.45833333  1.54166667 -0.375     ]
[ 2.64027778 -3.85277778  3.63333333 -1.76944444  0.34861111]
Adam-Moulton
[ 0.41666667  0.66666667 -0.08333333]
[ 0.375       0.79166667 -0.20833333  0.04166667]
[ 0.34861111  0.89722222 -0.36666667  0.14722222 -0.02638889]
Adam-Bashforth Coefficient for LTE
0.41666666666666663
0.375
0.34861111111111115
0.3298611111111111
Adam-Moulton Coefficient for LTE
-0.041666666666666664
-0.026388888888888906
-0.01875


### General Adams-Bashforth Methods

In [16]:
def AdamsBashforth(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
       a:Union[float, int],
       b:Union[float, int],
       y_init:Union[float, int],
       h:Union[float, int]=None,
       m:int=None,
       N:int=None,
       helper:callable=RK4,
       ) -> float:
    
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)

    coef = MultiStepCoef(m)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, ))
    w_[:m] = helper(f, a, b, y_init, h)[1][:m]
    
    for idx in range(m, N + 1):
        f_val = np.array([f(t_[idx - jdx], w_[idx - jdx]) for jdx in range(1, m + 1)])
        w_[idx] = w_[idx - 1] + h * (f_val * coef).sum()
    return (t_, w_)


In [17]:
f = lambda t, y: y - t ** 2 + 1
AdamsBashforth(f, 0, 2, 0.5, 0.2, 4)

(array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. ]),
 array([0.5       , 0.82929333, 1.21407621, 1.64892202, 2.12728925,
        2.64105333, 3.18031413, 3.73301859, 4.28444241, 4.81659556,
        5.30750818]))

### Predictor-Corrector Method

# 

In [18]:
def AdamsMoulton(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
                 f_:np.ndarray,
                 t:float,
                 w:float,
                 w_estimated:float,
                 h:float,
                 m:int
                ) -> float:
    '''
    For internal use of Predictor-Corrector Method only
    '''
    coef = MultiStepCoef(m, False)
    f_cur = f(t, w_estimated) * coef[0]
    return w + h * (f_cur + (f_ * coef[1:]).sum())

def PredictorCorrector(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
       a:Union[float, int],
       b:Union[float, int],
       y_init:Union[float, int],
       h:Union[float, int]=None,
       m:int=4,
       N:int=None,
       helper:callable=RK4
       ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)

    coef = MultiStepCoef(m)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, ))
    w_[:m] = helper(f, a, b, y_init, h)[1][:m]
    
    for idx in range(m, N + 1):
        f_val = np.array([f(t_[idx - jdx], w_[idx - jdx]) for jdx in range(1, m + 1)])
        w_estimated = w_[idx - 1] + h * (f_val * coef).sum()
        w_[idx] = AdamsMoulton(f, f_val[:m - 1], t_[idx], w_[idx - 1], w_estimated, h, m)
    return (t_, w_)


### Milne-Simpson Predictor-Corrector

In [19]:
def MilneSimpsonPC(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
       a:Union[float, int],
       b:Union[float, int],
       y_init:Union[float, int],
       h:Union[float, int]=None,
       N:int=None,
       helper:callable=RK4
       ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)


    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, ))
    w_[:4] = helper(f, a, b, y_init, h)[1][:4]
    
    for idx in range(4, N + 1):
        f_0, f_1 = f(t_[idx - 1], w_[idx - 1]), f(t_[idx - 2], w_[idx - 2])
        w_estimated = w_[idx - 4] + 4 * h * (2 * f_0 - f_1 + 2 * f(t_[idx - 3], w_[idx - 3])) / 3
        w_[idx] = w_[idx - 2] + h * (f(t_[idx], w_estimated) + 4 * f_0 + f_1) / 3
    return (t_, w_)

### Newton-Cotes Predictor-Corrector

In [20]:
# This function is used to form polynomial from roots which will be used in
# Newton-Cotes formula
def integPolynomial(index:int, roots:np.ndarray, coef:np.ndarray) -> float:
    idxs = np.arange(len(roots)) != index
    poly = np.polynomial.Polynomial.fromroots(roots[idxs])
    an_poly = np.polynomial.Polynomial.integ(poly)
    coef[index] = an_poly(1) - an_poly(0)
    coef[index] /= np.prod(roots[index] - roots[idxs])

def closedNewtonCotesCoef(num_points:int) -> np.ndarray:
    h = 1 / num_points
    num_points += 1
    roots = np.linspace(0, 1, num_points)
    coef = np.zeros(num_points, dtype=float)
    for idx in range(num_points):
        integPolynomial(idx, roots, coef)
    return coef / h

def openNewtonCotesCoef(num_points:int) -> np.ndarray:
    h = 1 / (num_points + 2)
    num_points += 1
    roots = np.linspace(h, 1 - h, num_points)
    coef = np.zeros(num_points, dtype=float)
    for idx in range(num_points):
        integPolynomial(idx, roots, coef)
    return coef / h

def NewtonCotesPC(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
       a:Union[float, int],
       b:Union[float, int],
       y_init:Union[float, int],
       h:Union[float, int]=None,
       m:int=2,
       N:int=None,
       helper:callable=RK4
       ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    
    openCoef = openNewtonCotesCoef(m)
    closedCoef = closedNewtonCotesCoef(m)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, ))
    w_[:m + 2] = helper(f, a, b, y_init, h)[1][:m + 2]
    
    for idx in range(m + 2, N + 1):
        f_val = np.array([f(t_[idx - jdx], w_[idx - jdx]) for jdx in range(m + 1, 0, -1)])
        w_estimated = w_[idx - (m + 2)] + np.sum(f_val * openCoef) * h
        w_[idx] = w_[idx - m] + (closedCoef[-1] * f(t_[idx], w_estimated) + np.sum(f_val[1:] * closedCoef[:m])) * h
    return (t_, w_)

In [21]:


f = [
    lambda t, y: y / t - (y / t) ** 2,
    lambda t, y: 1 + y / t + (y / t) ** 2,
    lambda t, y: -(y + 1) * (y + 3),
    lambda t, y: -5 * y + 5 * t ** 2 + 2 * t
]
a = [1, 1, 0, 0]
b = [2, 3, 2, 1]
y_init = [1, 0, -2, 1 / 3]
h = [0.1, 0.2, 0.1, 0.1]
actual = [
    1.18123222,
    5.87409998,
    -1.03597242,
    1.00224598
    ]

ms_pc = []
nw_pc_2 = []
nw_pc_3 = []
nw_pc_4 = []
nw_pc_5 = []
nw_pc_6 = []
nw_pc_7 = []
nw_pc_8 = []
nw_pc_9 = []
nw_pc_10 = []
pc = []

for idx in range(len(f)):
    ms_pc.append(MilneSimpsonPC(f[idx], a[idx], b[idx], y_init[idx], h[idx], helper=RK4)[1][-1])
    nw_pc_2.append(NewtonCotesPC(f[idx], a[idx], b[idx], y_init[idx], h[idx], m=2)[1][-1])
    nw_pc_3.append(NewtonCotesPC(f[idx], a[idx], b[idx], y_init[idx], h[idx], m=3)[1][-1])
    nw_pc_4.append(NewtonCotesPC(f[idx], a[idx], b[idx], y_init[idx], h[idx], m=4)[1][-1])
    nw_pc_5.append(NewtonCotesPC(f[idx], a[idx], b[idx], y_init[idx], h[idx], m=5)[1][-1])
    nw_pc_6.append(NewtonCotesPC(f[idx], a[idx], b[idx], y_init[idx], h[idx], m=6)[1][-1])
    nw_pc_7.append(NewtonCotesPC(f[idx], a[idx], b[idx], y_init[idx], h[idx], m=7)[1][-1])
    nw_pc_8.append(NewtonCotesPC(f[idx], a[idx], b[idx], y_init[idx], h[idx], m=8)[1][-1])
    nw_pc_9.append(NewtonCotesPC(f[idx], a[idx], b[idx], y_init[idx], h[idx], m=9)[1][-1])
    nw_pc_10.append(NewtonCotesPC(f[idx], a[idx], b[idx], y_init[idx], h[idx], m=10)[1][-1])
    pc.append(PredictorCorrector(f[idx], a[idx], b[idx], y_init[idx], h[idx], 4)[1][-1])

nw_pc = [nw_pc_2, nw_pc_3, nw_pc_4, nw_pc_5, nw_pc_6, nw_pc_7, nw_pc_8, nw_pc_9, nw_pc_10]
pd.DataFrame(
    {
        "Actual":actual,
        "Adams PC":pc,
        "Mile-Simpson":ms_pc,
        **{f"Newton-Cotes_{idx}":nw_pc[idx - 2] for idx in range(2, len(nw_pc) + 2)}
    }
)


,Actual,Adams PC,Mile-Simpson,Newton-Cotes_2,Newton-Cotes_3,Newton-Cotes_4,Newton-Cotes_5,Newton-Cotes_6,Newton-Cotes_7,Newton-Cotes_8,Newton-Cotes_9,Newton-Cotes_10
0,1.18123222,1.18121122,1.18122584,1.18122584,1.18122922,1.18123116,1.18123176,1.18123187,1.18123183,1.18123193,1.18123186,1.18123186
1,5.87409998,5.87395182,5.87375555,5.87375555,5.87382200,5.87402834,5.87399685,5.87400935,5.87400058,5.87396250,5.87383857,5.87383857
2,-1.03597242,-1.03597573,-1.03597247,-1.03597247,-1.03597093,-1.03597253,-1.03597266,-1.03597186,-1.03597282,-1.03597167,-1.03597135,-1.03597585
3,1.00224598,1.00213722,1.00215439,1.00215439,1.00278560,1.00214884,1.00227133,1.00233286,1.00220878,1.00204254,1.00232067,1.00232067


## Chapter 5.7 Variable Step-Size MultiStep Methods
The Runge-Kutta-Fehlberg method is used for error control because at each step it provides,
at little additional cost, two approximations that can be compared and related to the local
truncation error. Predictor-corrector techniques always generate two approximations at each
step, so they are natural candidates for error-control adaptation.

In [22]:
def VariablePredictorCorrector(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
       a:Union[float, int],
       b:Union[float, int],
       y_init:Union[float, int],
       h_max:Union[float, int],
       h_min:Union[float, int],
       m:int,
       n:int,
       delta_coef:float,
       helper:Callable,
       tol:float=1e-6,) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:

    AM_LTE = abs(MultiStepCoef(m + 1, False)[-1])
    sigma_coef = AM_LTE / (abs(MultiStepCoef(m + 1)[-1]) + AM_LTE)

    coef = MultiStepCoef(m)
    t_, w_ = [a], [y_init]
    h = h_max
    rk_computed = helper(f, h, t_, w_)
    flag = True
    last = False

    idx = m
    t = t_[-1] + h
    while flag:
        f_val = np.array([f(t_[idx - jdx], w_[idx - jdx]) for jdx in range(1, m + 1)])
        w_AB = w_[idx - 1] + h * (f_val * coef).sum()
        w_AM = AdamsMoulton(f, f_val[:m - 1], t, w_[idx - 1], w_AB, h, m)
        sigma = abs(w_AM - w_AB) * sigma_coef / h
        if sigma <= tol:
            t_.append(t)
            w_.append(w_AM)
            if last:
                flag = False
            else:
                idx += 1
                rk_computed = False
                if sigma <= 0.1 * tol or t_[idx - 1] + h > b:
                    delta = (tol / sigma) ** (1 / n) * delta_coef
                    h *= delta if delta <= 4 else 4
                    h = min(h, h_max)
                    if t_[idx - 1] + m * h > b:
                        h = (b - t_[idx - 1]) / m
                        last = True
                    rk_computed = helper(f, h, t_, w_)
                    idx += m - 1
        else:
            delta = (tol / sigma * delta_coef) ** (1 / n)
            h *= delta if delta >= 0.1 else 0.1
            if h < h_min:
                return (np.array(t_), np.array(w_))
            if rk_computed:
                idx -= m - 1
            t_ = t_[:idx]
            w_ = w_[:idx]
            rk_computed = helper(f, h, t_, w_)
            idx += m - 1
        t = t_[-1] + h
    return (np.array(t_), np.array(w_))

### Helper Runge-Kutta Methods function

In [23]:
def RK4_(f, h, t_, w_):
    x_0, y_0 = t_[-1], w_[-1]
    x_, y_ = [x_0] * 4, [y_0] * 4
    for idx in range(1, 4):
        K_1 = h * f(x_0, y_0)
        K_2 = h * f(x_0 + h / 2, y_0 + K_1 / 2)
        K_3 = h * f(x_0 + h / 2, y_0 + K_2 / 2)
        K_4 = h * f(x_0 + h, y_0 + K_3)
        y_0 += (K_1 + 2 * K_2 + 2 * K_3 + K_4) / 6
        x_0 += h
        x_[idx] = x_0
        y_[idx] = y_0
    t_.extend(x_[1:])
    w_.extend(y_[1:])
    return True

def RK5_(f, h, t_, w_):
    x_0, y_0 = t_[-1], w_[-1]
    x_, y_ = [x_0] * 5, [y_0] * 5
    for idx in range(1, 5):
        K_1 = h * f(x_0, y_0)
        K_2 = h * f(x_0 + h / 2, y_0 + K_1 / 2)
        K_3 = h * f(x_0 + h / 2, y_0 + K_1 / 4 + K_2 / 4)
        K_4 = h * f(x_0 + h, y_0 - K_2 + 2 * K_3)
        K_5 = h * f(x_0 + 2 * h / 3, y_0 + 7 * K_1 / 27 + 10 * K_2 / 27 + K_4 / 27)
        K_6 = h * f(x_0 + h / 5, y_0 + 28 * K_1 / 625 - K_2 / 5 + 546 * K_3 / 625 + 54 * K_4 / 625 - 378 * K_5 / 625)
        y_0 += (K_1/ 24 + 5 * K_4 / 48 + 27 * K_5 / 56 + 125 * K_6 / 336)
        x_0 += h
        x_[idx] = x_0
        y_[idx] = y_0
    t_.extend(x_[1:])
    w_.extend(y_[1:])
    return True

In [24]:
# Exercise 1, 2, 3 for Section 5.5 and 5.7
# Some precision error compared to the answers may due to that Python take in account more precision
# and causes the step size becomes slightly different

f = [
    lambda t, y: t * np.exp(3 * t) - 2 * y,
    lambda t, y: 1 + (t - y) ** 2,
    lambda t, y: 1 + y / t,
    lambda t, y: np.cos(2 * t) + np.sin(3 * t),

    lambda t, y: (y / t) ** 2 + y / t,
    lambda t, y: np.sin(t) + np.exp(-t),
    lambda t, y: (y ** 2 + y) / t,
    lambda t, y: -t * y + 4 * t / y,

    lambda t, y: y / t - (y / t) ** 2,
    lambda t, y: 1 + y / t + (y / t) ** 2,
    lambda t, y: -(y + 1) * (y + 3),
    lambda t, y: (t + 2 * t ** 3) * y ** 3 - t * y
]

a = [0, 2, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0]
b = [1, 3, 2, 1, 1.2, 1, 3, 1, 4, 3, 3, 2]
y_init = [0, 1, 2, 1, 1, 0, -2, 1, 1, 0, -2, 1 / 3]
h_max_multi = [0.25, 0.25, 0.25, 0.25, 0.05, 0.2, 0.4, 0.2, 0.5, 0.5, 0.5, 0.5]
h_min_multi = [0.025] * 4 + [0.01] * 4 + [0.02] * 4
h_max_rkf = [0.25, 0.25, 0.25, 0.25, 0.05, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
h_min_rkf = [0.05] * 4 + [0.02] * 4 + [0.05] * 4
tol = [10 ** (-4)] * 8 + [10 ** (-6)] * 4
actual=[
    3.2190993,
    2.5,
    5.3862944,
    2.1179795,
    1.46756957,
    1.09181825,
    -1.2,
    2.6666667,
    1.6762391,
    5.8741000,
    -1.0049452,
    0.0543455
]


pc_5 = []
pc_4 = []
rkf = []
for idx in range(len(f)):
    pc_5.append(VariablePredictorCorrector(f[idx], a[idx], b[idx], y_init[idx], h_max_multi[idx], h_min_multi[idx], 5, 5, 0.5, RK5_, tol[idx])[1][-1])
    pc_4.append(VariablePredictorCorrector(f[idx], a[idx], b[idx], y_init[idx], h_max_multi[idx], h_min_multi[idx], 4, 4, 0.5, RK4_, tol[idx])[1][-1])
    rkf.append(RKEmbedded(f[idx], a[idx], b[idx], RKF_rk, RKF_b, RKF_bp, RKF_c, y_init[idx], h_max_rkf[idx], h_min_rkf[idx], 4, 0.84, tol[idx])[1][-1])


pd.DataFrame(
    {
        "Actual":actual,
        "VariablePredictorCorrector Order 5":pc_5,
        "VariablePredictorCorrector Order 4":pc_4,
        "RKF":rkf
    }
)

C:\Users\User\AppData\Local\Temp\ipykernel_5540\3416140393.py:29: RuntimeWarning: invalid value encountered in scalar divide
  sigma = abs(w_AM - w_AB) * sigma_coef / h


,Actual,VariablePredictorCorrector Order 5,VariablePredictorCorrector Order 4,RKF
0,3.21909930,3.21910032,3.21912536,3.21904974
1,2.50000000,2.49998285,2.50001015,2.50000052
2,5.38629440,5.38629143,5.38627253,5.38629580
3,2.11797950,2.11796122,2.11801281,2.11797499
4,1.46756957,1.46756952,1.46756885,1.46756971
5,1.09181825,1.09181857,1.09183759,1.09181816
6,-1.20000000,-1.20000249,-1.19999818,-1.20001945
7,2.66666670,1.70186484,1.70187341,1.70188143
8,1.67623910,1.67623919,1.67623904,1.67623925
9,5.87410000,5.87409979,5.87410206,5.87410585


## Chapter 5.8 Extrapolation Methods

In [25]:
def Extrapolation(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
       a:Union[float, int],
       b:Union[float, int],
       y_init:Union[float, int],
       h_min:float,
       h_max:float,
       tol:float=1 ** (-10),
       nodes:Union[np.ndarray[int, Any], List[int]]=[2, 4, 6, 8, 12, 16, 24, 32]
    ):
   h = h_max
   midpoint = lambda w_, t, w: w_ + 2 * h * f(t, w)
   euler = lambda t, w: w + h * f(t, w) / 2
   w_1 = euler(a, y_init)
 


    

## Chapter 5.9 Higher-Order Equations and Systems of Differential Equations

In [54]:
def RK4_System(f:List[Callable], a:Union[float, int], b:Union[float, int], 
        y_init:Union[List[Union[float, int]], np.ndarray],
        h:Union[float, int, None]=None,
        N:Union[int, None]=None,
        ):

    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)    
    t_ = np.linspace(a, b, N + 1) 
    k = np.zeros((4, len(f)))
    w_ = np.zeros((N + 1, len(f)))
    w_[0, :] = y_init.copy()
    for idx in range(N):
        for jdx in range(len(f)):
            k[0, jdx] = h * f[jdx](t_[idx], w_[idx, :])
        for jdx in range(len(f)):
            k[1, jdx] = h * f[jdx](t_[idx] + h / 2, w_[idx, :] + k[0, :] / 2)
        for jdx in range(len(f)):
            k[2, jdx] = h * f[jdx](t_[idx] + h / 2, w_[idx, :] + k[1, :] / 2)  
        for jdx in range(len(f)):
            k[3, jdx] = h * f[jdx](t_[idx] + h, w_[idx, :] + k[2, :])  
        w_[idx + 1, :] = w_[idx, :] + (k[0, :] + 2 * k[1, :] + 2 * k[2, :] + k[3, :]) / 6
    return (t_, w_)


In [55]:
# Exercise 1
f = [
        lambda t, w: 3 * w[0] + 2 * w[1] - (2 * t ** 2 + 1) * np.exp(2 * t),
        lambda t, w: 4 * w[0] + w[1] + (t ** 2 + 2 * t - 4) * np.exp(2 * t),
    ]
a, b = 0, 1
h = 0.2
alpha = [1, 1]

RK4_System(f, a, b, alpha, h)

(array([0. , 0.2, 0.4, 0.6, 0.8, 1. ]),
 array([[ 1.        ,  1.        ],
        [ 2.12036583,  1.50699185],
        [ 4.44122776,  3.24224021],
        [ 9.73913329,  8.163417  ],
        [22.67655977, 21.34352778],
        [55.66118088, 56.03050296]]))

## Chapter 5.11 Stiff Differential Equations

In [67]:
def TrapezoidalNewton(
        f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
        fy:Callable[[Union[float, int], Union[float, int]], Union[float, int]], 
        a:Union[float, int],
        b:Union[float, int],
        alpha:Union[float, int],
        h:Union[float, int]=None,
        N:int=None,
        tol:float=1e-5,
        M:int=10
        ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * alpha
    for idx in range(1, N + 1):
        k = w_[idx - 1] + h * f(t_[idx - 1], w_[idx - 1]) / 2
        w = k
        for _ in range(M):
            w_[idx] = w - (w - h * f(t_[idx - 1] + h, w) / 2 - k) / (1 - h * fy(t_[idx - 1] + h, w) / 2)
            if np.abs(w_[idx] - w) < tol:
                break
            else:
                w = w_[idx]
    return (t_, w_)

In [68]:
# Illustration
f = lambda t, w: 5 * np.exp(5 * t) * (w - t) ** 2 + 1
fy = lambda t, w: 10 * np.exp(5 * t) * (w - t)
a, b = 0, 1
alpha = -1

rk4_1 = []
rk4_2 = []
trap_newton_1 = []
trap_newton_2 = []


rk4_1.append(RK4(f, a, b, alpha, 0.2)[1][-1])
rk4_2.append(RK4(f, a, b, alpha, 0.25)[1][-1])
trap_newton_1.append(TrapezoidalNewton(f, fy, a, b, alpha, 0.2, tol=10 ** -6)[1][-1])
trap_newton_2.append(TrapezoidalNewton(f, fy, a, b, alpha, 0.25, tol=10 ** -6)[1][-1])


pd.DataFrame({
    "RK4, h=0.2":rk4_1,
    "RK4, h=0.25":rk4_2,
    "Trapezoidal Newton, h=0.2":trap_newton_1,
    "Trapezoidal Newton, h=0.25":trap_newton_2
})


C:\Users\User\AppData\Local\Temp\ipykernel_5540\1384223272.py:2: RuntimeWarning: overflow encountered in scalar power
  f = lambda t, w: 5 * np.exp(5 * t) * (w - t) ** 2 + 1


,"RK4, h=0.2","RK4, h=0.25","Trapezoidal Newton, h=0.2","Trapezoidal Newton, h=0.25"
0,0.99349051,inf,0.99377255,0.99401991


In [69]:
# Exercise 1
f = [
    lambda t, w: -9 * w,
    lambda t, w: -20 * (w - t ** 2) + 2 * t,
    lambda t, w: -20 * w + 20 * np.sin(t) + np.cos(t),
    lambda t, w: 50 / w - 50 * w
]

fy = [
    lambda t, w: -9,
    lambda t, w: -20,
    lambda t, w: -20,
    lambda t, w: -50 / w ** 2 - 50,
]

a = [0, 0, 0, 0]
b = [1, 1, 2, 1]
alpha = [np.exp(1), 1 / 3, 1, np.sqrt(2)]
h = [0.1, 0.1, 0.25, 0.1]

euler = []
rk4 = []
adams4_pc = []
trap_newton = []

for idx in range(len(f)):
    euler.append(Euler(f[idx], a[idx], b[idx], alpha[idx], h[idx])[1][-1])
    rk4.append(RK4(f[idx], a[idx], b[idx], alpha[idx], h[idx])[1][-1])
    adams4_pc.append(PredictorCorrector(f[idx], a[idx], b[idx], alpha[idx], h[idx], 4)[1][-1])
    trap_newton.append(TrapezoidalNewton(f[idx], fy[idx], a[idx], b[idx], alpha[idx], h[idx])[1][-1])


pd.DataFrame({
    "Euler":euler,
    "RK4":rk4,
    "Adams 4th Predictor Corrector":adams4_pc,
    "Trapezoidal Newton":trap_newton
})


,Euler,RK4,Adams 4th Predictor Corrector,Trapezoidal Newton
0,2.71828183e-10,3.72387642e-04,2.36042255e-03,0.00016759
1,1.33333333e+00,1.00250560e+00,7.27855650e-01,1.00000000
2,6.55231245e+04,1.24641320e+09,2.11574130e+09,0.91053267
3,3.87332001e+05,-2.69031268e+11,-5.66751172e+11,-1.00284456
